**Group 30**\
Emre Kaplaner: 13884557\
Ionel Popescu: 13899430\
Piotr Zielinski: 13747088

# <font color=red>(Part A) Transportation problem</font>


## Implementation in Python:

In [1]:
from gurobipy import *

def TransportationModel(supply, demand, cost):

    depots    = range(len(supply))         # Indices for the depots
    customers = range(len(demand))         # Indices for the customers

    if sum(supply) < sum(demand):
        print("Not balanced: supply (", sum(supply), ") < demand (", sum(demand), ")")
        return Model()                                                             # Unbalanced problems are not solved
    elif sum(supply) > sum(demand):
        print("Not balanced: supply (", sum(supply), ") > demand (", sum(demand), ")")
        return Model()                                                             # Unbalanced problems are not solved
    
    m = Model("Transportation")
    x = m.addVars(depots, customers, name = "Flow")          # no need to define as INTEGERs (see week 3)
    
    m.setObjective(quicksum(cost[i,j]*x[i,j] for i in depots for j in customers), GRB.MINIMIZE)
    m.addConstrs( (x.sum(i, "*") == supply[i] for i in depots), 'Supply')
    m.addConstrs( (x.sum("*", j) == demand[j] for j in customers), 'Demand')
    m.update()
         
    return m

## Use the model to solve the problem instance from the textbook:

In [13]:
import numpy as np

m = 3                                                                            # A small problem instance
n = 5
a = [8, 5, 6]                                                                    # Parameter a: supply
b = [2, 3, 5, 2, 7]                                                              # Parameter b: demand
c = np.array([ [4, 3, 8, 7, 9], [6, 5, 6, 4, 5], [7, 4, 7, 5, 4] ])              # Parameter c: unit cost matrix

TP = TransportationModel(a, b, c)
TP.Params.LogToConsole = 0
print(f'Model has {TP.NumVars} variables, {TP.NumConstrs} constraints and {TP.NumNZs} nonzeros\n')

TP.optimize()

status = TP.Status
if status != GRB.OPTIMAL:
    print(f"Optimization was stopped with status {status}")
else:
    print(f"The optimal objective is {TP.ObjVal:g}")
    print("\nVarName           |   value      c_ij    red.cost") 
    for var in TP.getVars():
        #if var.x > 0.5:
            print(f"{var.VarName:17} |{var.X:8.3f}  {var.obj:8.2f}   {var.rc:8.2f}")
    print("\nConstrName        |     RHS     Slack         Pi") 
    for cons in TP.getConstrs():
        print(f"{cons.ConstrName:17} |{cons.rhs:8.2f}  {cons.Slack:8.2f}   {cons.pi:8.2f}") 

Model has 15 variables, 8 constraints and 30 nonzeros

The optimal objective is 90

VarName           |   value      c_ij    red.cost
Flow[0,0]         |   2.000      4.00       0.00
Flow[0,1]         |   3.000      3.00       0.00
Flow[0,2]         |   3.000      8.00       0.00
Flow[0,3]         |   0.000      7.00       1.00
Flow[0,4]         |   0.000      9.00       2.00
Flow[1,0]         |   0.000      6.00       4.00
Flow[1,1]         |   0.000      5.00       4.00
Flow[1,2]         |   2.000      6.00       0.00
Flow[1,3]         |   2.000      4.00       0.00
Flow[1,4]         |   1.000      5.00       0.00
Flow[2,0]         |   0.000      7.00       6.00
Flow[2,1]         |   0.000      4.00       4.00
Flow[2,2]         |   0.000      7.00       2.00
Flow[2,3]         |   0.000      5.00       2.00
Flow[2,4]         |   6.000      4.00       0.00

ConstrName        |     RHS     Slack         Pi
Supply[0]         |    8.00      0.00       2.00
Supply[1]         |    5.00     

## Problem W2.0

As follow-up questions on the problem solved above: write a few line of code that produce: \
   - the objective value of the dual (z_dual) and compares the value with the objective value of the primal\
   - use the duals to calculate the reduced cost for the basic variables. (Are they all equal to 0?)\
   - use the duals to calculate the reduced cost for the non-basic variables. (Are they all < 0?)

In [14]:
import numpy as np
from itertools import product

# Extract the primal solution x
x = np.array([var.x for var in TP.getVars()]).reshape(m,n)   

# Extract the dual solution u and v
u, v = np.array([cons.pi for cons in TP.getConstrs()][0:m]), np.array([cons.pi for cons in TP.getConstrs()][m:m+n])
print(u,v)

#Compute objective value of dual (z_dual)
z_dual = np.dot(a, u) + np.dot(b, v) # b transpose (RHS of the constraints) * y_dual (dual solution: [u v])
print(f"The objective value of the dual is {z_dual}")










[ 2.  0. -1.] [2. 1. 6. 4. 5.]
The objective value of the dual is 90.0


# <font color=red>(Part B) Use the model with all kinds of data</font>

As in real life, most transportation problems are not balanced, can formulate to similar models: one for problems with total supply > total demand and one for problems with total supply < total demand. Alternatively, we can just simply adjust the data in case we encounter an unbalanced problem. That is, add either a dummy supply node or a dummy demand node.
- (A) to study the balanced version of the Transportation Problem, 
- (B) use it for an unbalanced problem with data being read from an Excel file,
- (C) and use it for a coaching problem similar to the Linear Assignment Problem. More concrete, four swimmers need to be selected for the three 4x100m Medley teams (Womens, Mens and Mixed team). See if you select the gold winning team from the last European Championship.

## Problem W2.1
In the cell below, data is read from an Excel-file "Transportation.xlsx" for an unbalanced Transportation Problem. Running the cell, you will notice that it will not yield a solution. Modify the code below (not the TransportationModel() function) in order to get an equivalent balanced problem instance and present the solution for the original problem instance.

In [4]:
import pandas as pd
import numpy as np

m = 5
n = 5
c = pd.read_excel("Transportation.xlsx", sheet_name = 'Example', header = None, skiprows = 1, nrows = m, usecols = 'B:F').values.tolist()
a = pd.read_excel("Transportation.xlsx", sheet_name = 'Example', header = None, skiprows = 1, nrows = m, usecols = 'G').values.flatten().tolist()
b = pd.read_excel("Transportation.xlsx", sheet_name = 'Example', header = None, skiprows = m+1, nrows = 1, usecols = 'B:F').values.tolist()[0] 

### Adjust the parameters in order to have a balanced problem that can be solved with TransportationModel()

#We add a dummy demand node = 2 s.t. supply equal demand 
b.append(2)
# Add a 0 cost column at the end of the cost matrix
c = np.array(c)
c = np.append(c, np.zeros((5,1)), axis = 1)

print("Cost matrix:")
print(c)
print("Supply:", a)
print("Demand:", b)

TP = TransportationModel(a, b, c)
TP.Params.LogToConsole = 0


print(f'Model has {TP.NumVars} variables, {TP.NumConstrs} constraints and {TP.NumNZs} nonzeros\n')

TP.optimize()

print(f"The optimal objective is {TP.ObjVal:g}")
for var in TP.getVars():
   if abs(var.x) > 1e-6: # only printing non-zeros
       print('{0} = {1:6.2f}'.format(var.VarName[:18], var.x))

Cost matrix:
[[ 13.  10.  22.  29.  18.   0.]
 [ 14.  13.  16.  21. 999.   0.]
 [  3.   0. 999.  11.   6.   0.]
 [ 18.   9.  19.  23.  11.   0.]
 [ 30.  24.  34.  36.  28.   0.]]
Supply: [5, 6, 7, 4, 3]
Demand: [3, 5, 4, 5, 6, 2]
Model has 30 variables, 11 constraints and 60 nonzeros

The optimal objective is 276
Flow[0,0] =   3.00
Flow[0,1] =   2.00
Flow[1,2] =   4.00
Flow[1,3] =   2.00
Flow[2,1] =   3.00
Flow[2,3] =   3.00
Flow[2,4] =   1.00
Flow[3,4] =   4.00
Flow[4,4] =   1.00
Flow[4,5] =   2.00


This solution considered the dummy demand node number 6 with a demand of 2 that was met with a cost of 0. Thus, the results of this LO model can still be used to answer the original unbalanced problem. 

# <font color=red>(Part C) Team selection for the 4x100m Medley </font>

## Problem W2.2
In the same Excel file "Transportation.xlsx", the Personal Records of Dutch swimmers who took part in the European Championships of 2022 are provided. Your task is to form teams for the 4x100m relay (Women, Men and Mixed) based on their PR's, i.e. four individuals are selected such that the total PR-time is minimal.

In the cell below, the Women's team is already determined. Adjust the code in order to find the Men's team and the Mixed team. The latter should consist of two male and two female swimmers (check if you have found the winning team).

Note: the problems could be solved by hand rather easily, but that would not be very interesting. Besides that, in reality also reserve-swimmers can be selected for the qualifying heats and the schedule of the championship should be taken in consideration in order to avoid overloading certain swimmers.

In [5]:
import pandas as pd
from datetime import datetime

###WOMAN

m = 13                # Number of Women in TeamNL
n = 5                 # The 5-th event is being a supporter
times_w = pd.read_excel("Transportation.xlsx", sheet_name = 'Medley', header = None, skiprows = 4, nrows = m, usecols = 'B:E').to_numpy()
times_m= pd.read_excel("Transportation.xlsx", sheet_name = 'Medley', header = None, skiprows = 20, nrows = (m-3), usecols = 'B:E').to_numpy() 

times_3=np.r_[times_w,times_m]
I = range(m)
J = range(4)
    
a = [1 for i in range(m)]
b = [1 for i in range(n)]
b[n-1] = m - n + 1
c = [[ (60*times_w[i][j].minute + times_w[i][j].second + times_w[i][j].microsecond/1e6) for j in range(n-1)] for i in range(m)] 
[c[i].append(0) for i in range(m)]
c = np.array(c)
print('Woman')
print("PB's on the four events")
print(c)
print("Supply:", a, "  Demand", b)

TP = TransportationModel(a, b, c)
TP.Params.LogToConsole = 0
print(f'Model has {TP.NumVars} variables, {TP.NumConstrs} constraints and {TP.NumNZs} nonzeros\n')

TP.optimize()
sol_wom=[]
for j in J:
    for i in I:
        xij = TP.getVarByName(f"Flow[{i},{j}]")
        if xij.x == 1:
            sol_wom.append(c[i,j])
            print("Swimmer {:2}   on event {}   PR = {}".format(i+1, j+1, c[i,j]))
            
###MAN
            
m2=10
I2 = range(m2)
J2 = range(4)
a2 = [1 for i in range(m2)]
b2 = [1 for i in range(n)]
b2[n-1] = m2 - n + 1
c2 = [[ (60*times_m[i][j].minute + times_m[i][j].second + times_m[i][j].microsecond/1e6) for j in range(n-1)] for i in range(m2)] 
[c2[i].append(0) for i in range(m2)]
c2= np.array(c2)
print("Man")            
print("PB's on the four events")
print(c2)
print("Supply:", a2, "  Demand", b2)

TP = TransportationModel(a2, b2, c2)
TP.Params.LogToConsole = 0
print(f'Model has {TP.NumVars} variables, {TP.NumConstrs} constraints and {TP.NumNZs} nonzeros\n')

TP.optimize()
sol_man=[]
for j in J2:
    for i in I2:
        xij = TP.getVarByName(f"Flow[{i},{j}]")
        if xij.x == 1:
            sol_man.append(c2[i,j])
            print("Swimmer {:2}   on event {}   PR = {}".format(i+1, j+1, c2[i,j]))
# for var in TP.getVars():
#    if abs(var.x) > 1e-6: # only printing non-zeros
#        print('{0} = {1:6.2f}'.format(var.VarName[:18], var.x))

###MIXED

m3=23
n=6
I3 = range(m3)
J3 = range(4)
a3 = [1 for i in range(m3)]
b3 = [1 for i in range(n)]
b3[n-1] = m2 - 2
b3[n-2] = m - 2
c3 = [[ (60*times_3[i][j].minute + times_3[i][j].second + times_3[i][j].microsecond/1e6) for j in range(5-1)] for i in range(m3)] 
[c3[i].append(0) for i in range(m)]
[c3[i+m].append(10) for i in range(m2)]
[c3[i].append(10) for i in range(m)]
[c3[i+m].append(0) for i in range(m2)]

c3= np.array(c3)
print("Mixed")            
print("PB's on the four events")
print(c3)
print("Supply:", a3, "  Demand", b3)

TP = TransportationModel(a3, b3, c3)
TP.Params.LogToConsole = 0
print(f'Model has {TP.NumVars} variables, {TP.NumConstrs} constraints and {TP.NumNZs} nonzeros\n')

TP.optimize()
sol_mixed=[]
for j in J3:
    for i in I3:
        xij = TP.getVarByName(f"Flow[{i},{j}]")
        if xij.x == 1:
            sol_mixed.append(c3[i,j])
            print("Swimmer {:2}   on event {}   PR = {}".format(i+1, j+1, c3[i,j]))
# for var in TP.getVars():
#    if abs(var.x) > 1e-6: # only printing non-zeros
#        print('{0} = {1:6.2f}'.format(var.VarName[:18], var.x))


Woman
PB's on the four events
[[63.91 74.86 54.6  69.96  0.  ]
 [58.65 90.89 54.48 64.26  0.  ]
 [59.62 81.14 55.99 58.1   0.  ]
 [65.56 89.02 54.57 58.71  0.  ]
 [63.25 72.05 53.24 59.3   0.  ]
 [79.77 66.92 68.3  74.43  0.  ]
 [75.37 69.75 54.05 59.12  0.  ]
 [69.76 69.19 58.29 62.64  0.  ]
 [70.44 83.23 55.47 64.38  0.  ]
 [64.7  76.5  55.69 62.21  0.  ]
 [68.24 83.03 55.35 64.75  0.  ]
 [65.94 89.48 55.83 65.19  0.  ]
 [61.35 74.5  58.33 60.44  0.  ]]
Supply: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]   Demand [1, 1, 1, 1, 9]
Model has 65 variables, 18 constraints and 130 nonzeros

Swimmer  2   on event 1   PR = 58.65
Swimmer  6   on event 2   PR = 66.92
Swimmer  5   on event 3   PR = 53.24
Swimmer  3   on event 4   PR = 58.1
Man
PB's on the four events
[[ 70.45  57.8   53.74  54.74   0.  ]
 [ 68.65  83.2   49.42  55.02   0.  ]
 [ 56.95  87.53  48.99  55.71   0.  ]
 [ 58.16  63.4   48.53  60.46   0.  ]
 [ 65.92  66.33  50.48  53.16   0.  ]
 [ 55.48  74.3   48.86  51.41   0.  ]
 [ 56.8

# <font color=red>(Part D) Add your own models</font>

## Now, solve your models for problem W2.3 ... W2.5

## W2.3
### a

We first consider this model by address the following:\
 - $x_1 \in \mathbb{R}$ : when the 4th constraint is imposed, it implicitly accounts for the non negativity of $x_1$\
 - $x_3 \geq 0$: $x_3 = -x_3^{'}$\
 - we use slack variables and the big M method to address the $\geq$- constraints\
 
 Then, the model becomes:\
 
 max $z = (2+2M)y_1 + (3+6M)y_2+ (3+2M)y_3^{'} + (1+5M)y_4 - M y_5 - M y_7 - My_8 - 16M$\
 s.t. $2y_2 +y_3 + 3 y_4 + y_5 + y_6 = 8$\
 &emsp; $y_1 +y_2 + 2y_3 +y_4 + a_1 = 5$\
 &emsp; $5y_2 + 4y_4 - y_5 - y_7 + a_2 = 10$\
 &emsp; $y_1  -y_8 + a_3 = 1$\
 &emsp; $y_1, y_2, y'_3, y_4, y_5, y_6, y_7, y_8, a_1, a_2, a_3 \geq 0$\

 And the corresponding initial simplex tableau is : 
 
 
 

|     |     | $y_1$| $y_2$ |$y'_3$|$y_4$|$y_5$|$y_6$|$y_7$|$y_8$|$a_1$| $a_2$ | $a_3$ |-z|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| **$c_B$**   |  **$x_B$** |  2+2M  | 3+6M |3+2M|  1+5M |  -M|  0 |  -M |  -M | 0 |  0 | 0 | **16M** |
| 0 | $y_6$ | 0 | 2 | 1 | 3 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | **8** |
| 0 | $a_1$ | 1 | 1 | 2 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | **5** |
| 0 | $a_2$ | 0 |  5 | 0 | 4 | -1 | 0 | 1 | 0 | 0 | 1 | 0 | **10** |
| 0 | $a_3$ | 1 |0 | 0 | 0 | 0 | 0 | 0 | -1 | 0 | 0 | 1 | **1** |

### b.

From constraint 3 we choose to express $x_1$ as a function of the other variables. Plugging this is the first model we get the following LO model: 

max $z = (1+5M)y_2 - y_3^{'} + (4M - 1)y_4 - M y_5 - M y_7 +10 - 10M$\
s.t. $2y_2 +y_3^{'} + 3 y_4 + y_5 + y_6 = 8$\
&emsp; $5y_2 + 4y_4 - y_5 - y_7 + a_1 = 10$ \
&emsp; $y_2 + 2y_3^{'} +y_4+ y_8 = 4$\
&emsp; $y_2, y_3^{'}, y_4, y_5, y_6, y_7, y_8, a_1 \geq 0$
<br>

And the corresponding initial simplex tableau is :

|  |  |$y_2$ | $y'_3$ | $y_4$ | $y_5$ | $y_6$ | $y_7$ | $y_8$ | $a_1$  | -z |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|  $c_B$   |  $x_B$ |  1+5M  |  -1 |  4M-1 |  -M |  0 |  -M |  0 |  0 | 10M-10 |
| 0 | $y_6$ | 2 | 1 | 3 | 1 | 1 | 0 | 0 | 0 | 8 |
| 0 | $a_1$ | 5 | 0 | 4 | -1 | 0 | -1 | 0 | 1 | 10 |
| 0 | $y_8$| 1 | 2 | 1| 0 | 0 | 0 | 1| 0 | 4 |



### c.

In [45]:
m = Model("W2.3")
y = m.addVars(5)
m.setObjective(2*y[0] +3*y[1] - 3*y[2] + y[3], GRB.MAXIMIZE)

m.addConstr(2*y[1] - y[2] + 3*y[3] + y[4] <= 8)
m.addConstr(y[0] + y[1] - 2*y[2] + y[3] == 5)
m.addConstr(5*y[1] +4*y[3]-y[4] >=10)
m.addConstr(y[0] >=1)
m.addConstr(y[2] <= 0)

m.optimize()

print(f"The optimal objective is {m.ObjVal:g}")
print("\nVarName           |   value      c_ij    red.cost") 
for var in m.getVars():
     print(f"{var.VarName:17} |{var.X:8.3f}  {var.obj:8.2f}   {var.rc:8.2f}")
print("\nConstrName        |     RHS     Slack         Pi") 
for cons in m.getConstrs():
        print(f"{cons.ConstrName:17} |{cons.rhs:8.2f}  {cons.Slack:8.2f}   {cons.pi:8.2f}") 

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1030NG7 CPU @ 1.10GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 5 columns and 13 nonzeros
Model fingerprint: 0x437e2b20
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 2 rows and 2 columns
Presolve time: 0.04s
Presolved: 3 rows, 3 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4000000e+01   0.000000e+00   0.000000e+00      0s
       0    1.4000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.05 seconds (0.00 work units)
Optimal objective  1.400000000e+01
The optimal objective is 14

VarName           |   value      c_ij    red.cost
C0                |   1.000      2.00       0.00
C1                |   4

By using Gurobi Optimizer we obtain the optimal objective value of 14 and $x^* = [3,4,0,0]^T$. As we can see, in the Presolve phase, Gurobi removed 2 rows and 2 columns. This accounts for the difference of 4 variables between the two simplex tableaux discused above. 

## W2.4 

### a. 

**Decision variables**\
&emsp; $x_1 =$ number of 4 rooms houses \
&emsp; $x_2 =$ number of 6 rooms houses \
&emsp; $x_3 =$ number of 6 bedrooms luxury houses

**Objective function (x 10000):**\
&emsp; max $z = 180 x_1 + 300 x_2 + 360 x_3$ 

**Subject to the following constraints:**\
&emsp; $x1+x2+x3 \geq 60$\
&emsp; $4x1+6x2+7x3\geq270$  (We assume that for luxury houses the number of rooms is 7 (6 bedrooms + 1 living room)\
&emsp; $160x1+180x2+240x3 \leq 12900$\
&emsp; $12x1+20x2+24x3 \leq 1200$\
&emsp; $x_1,x_2,x_3 \geq 0$ for $i \in {1,2,3}$

### b. 

**The dual model:**
<br> 

min $z=60y_1+270y_2+12900y_3+1200y_4$\
s.t.$y_1+4y_2+160y_3+12y_4\geq180$\
&emsp;$y_1+6y_2+180y_3+20y_4\geq300$\
&emsp;$y_1+6y_2+200y_3+24y_4\geq360$\
&emsp;$y_1\leq0$,$y_2\leq0$,$y_3\geq0$,$y_4\geq0$




In [48]:
# Code for solving W2.4c
from gurobipy import *

# Create a Gurobi model
model = Model("Primal_LP")

# Decision Variables
x1 = model.addVar(lb=0, name="x1")
x2 = model.addVar(lb=0, name="x2")
x3 = model.addVar(lb=0, name="x3")

# Objective Function
model.setObjective(180000*x1 + 300000*x2 + 360000*x3, GRB.MAXIMIZE)

# Constraints
model.addConstr(x1 + x2 + x3 >= 60, "c1")
model.addConstr(4*x1 + 6*x2 + 7*x3 >= 270, "c2") #Luxury house: 6 bedrooms + 1 living room
model.addConstr(160*x1 + 180*x2 + 240*x3 <= 12900, "c3")
model.addConstr(120000*x1 + 200000*x2 + 240000*x3 <= 12000000, "c4")

# Solve the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Objective Value: {model.objVal}")
    print("\nOptimal Solution:")
    for var in model.getVars():
        print(f"{var.VarName}: {var.x}")
else:
    print(f"Optimization ended with status {model.status}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1030NG7 CPU @ 1.10GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 3 columns and 12 nonzeros
Model fingerprint: 0xfb0323f7
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [2e+05, 4e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 1e+07]
Presolve time: 0.01s
Presolved: 4 rows, 3 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6406250e+33   4.401855e+30   1.640625e+03      0s
       2    1.8000000e+07   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.800000000e+07
Optimal Objective Value: 18000000.0

Optimal Solution:
x1: 0.0
x2: 60.0
x3: 0.0


Our optimal solution indicates that the only standard 6 room houses should be built. 
<br>

### d. 
To the original primal we add a constraint:
$x_3 \leq \frac12(x_1+x_2+x_3)$

Since the optimal solution does not violate the new constraints, the optimal solution will remain the same. 

In [49]:
# Code for solving W2.4d
from gurobipy import *

# Create a Gurobi model
model = Model("Primal_LP_New_Regulation")

# Decision Variables
x1 = model.addVar(lb=0, name="x1")
x2 = model.addVar(lb=0, name="x2")
x3 = model.addVar(lb=0, name="x3")

# Objective Function
model.setObjective(180000*x1 + 300000*x2 + 360000*x3, GRB.MAXIMIZE)

# Constraints
model.addConstr(x1 + x2 + x3 >= 60, "c1")
model.addConstr(4*x1 + 6*x2 + 7*x3 >= 270, "c2")
model.addConstr(160*x1 + 180*x2 + 240*x3 <= 12900, "c3")
model.addConstr(120000*x1 + 200000*x2 + 240000*x3 <= 12000000, "c4")
model.addConstr(x3 <= 0.5*(x1 + x2 + x3), "new_regulation")

# Solve the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Objective Value: {model.objVal}")
    print("\nOptimal Solution:")
    for var in model.getVars():
        print(f"{var.VarName}: {var.x}")
else:
    print(f"Optimization ended with status {model.status}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1030NG7 CPU @ 1.10GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 3 columns and 15 nonzeros
Model fingerprint: 0xe2769594
Coefficient statistics:
  Matrix range     [5e-01, 2e+05]
  Objective range  [2e+05, 4e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 1e+07]
Presolve time: 0.01s
Presolved: 5 rows, 3 columns, 15 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6406250e+33   4.401855e+30   1.640625e+03      0s
       2    1.8000000e+07   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.800000000e+07
Optimal Objective Value: 18000000.0

Optimal Solution:
x1: 0.0
x2: 60.0
x3: 0.0


## Problem W2.5

### a. 
Our LO model for this problem is:

min $d$\
s.t.\
&emsp;$x_A+t_A\leq x_C$\
&emsp;$x_B+t_B\leq x_D$\
&emsp;$x_B+t_B\leq x_E$\
&emsp;$x_E+t_E\leq x_F$\
&emsp;$x_E+t_E\leq x_G$\
&emsp;$x_E+t_E\leq x_H$\
&emsp;$x_D+t_D\leq x_I$\
&emsp;$x_F+t_F\leq x_I$\
&emsp;$x_C+t_C\leq x_J$\
&emsp;$x_C+t_C\leq x_K$\
&emsp;$x_H+t_H\leq x_L$\
&emsp;$x_G+t_G\leq x_M$\
&emsp;$x_I+t_I\leq x_M$\
&emsp;$x_J+t_J\leq x_M$\
&emsp;$x_i+t_i\leq d$ for $i \in \{A,B,C,D,E,F,G,H,I,J,K,L,M\}$\
where $t_i$ is time duration of process $i$.


In [50]:
# Code for solving W2.5
from gurobipy import *

def ProjectSchedulingProblem(duration, pred):

    # Create a Gurobi model
    model = Model("Project_Scheduling")

    # Decision Variables
    start_times = {}
    for activity in duration:
        start_times[activity] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"start_{activity}")
    #last process variable
    d = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"Last_Process_start")

    # Objective Function: Minimize the total duration
    model.setObjective(d, GRB.MINIMIZE)

    # Constraints: Activity start times and immediate predecessors
    for activity in duration:
        model.addConstr(start_times[activity] >= 0, f"non_negative_{activity}")
        for predecessor in pred.get(activity, []):
            model.addConstr(start_times[activity] >= start_times[predecessor] + duration[predecessor], f"precedence_{activity}_{predecessor}")

    # Constraints: the last process
        for activity in duration:
            model.addConstr(start_times[activity]+duration[activity]<=d)

    # Solve the model
    model.optimize()

    # Extract results
    optimal_schedule = {activity: start_times[activity].x for activity in duration}
    optimal_duration = model.objVal

    return optimal_schedule, optimal_duration

# Example usage:
duration = {"A": 2, "B": 3, "C": 4, "D": 2, "E": 3, "F": 4, "G": 6, "H": 4, "I": 3, "J": 4, "K": 6, "L": 2, "M": 1}
predecessors = {"C": ["A"], "D": ["B"], "E": ["B"], "F": ["E"], "G": ["E"], "H": ["E"], "I": ["D", "F"], "J": ["C"], "K": ["C"], "L": ["H"], "M": ["G", "I", "J"]}

optimal_schedule, optimal_duration = ProjectSchedulingProblem(duration, predecessors)
print("Optimal Schedule:", optimal_schedule)
print("Optimal Duration:", optimal_duration)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i5-1030NG7 CPU @ 1.10GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 196 rows, 14 columns and 379 nonzeros
Model fingerprint: 0x884c73bc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+00]
Presolve removed 196 rows and 14 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.400000000e+01
Optimal Schedule: {'A': 0.0, 'B': 0.0, 'C': 2.0, 'D': 3.0, 'E': 3.0, 'F': 6.0, 'G': 6.0, 'H': 6.0, 'I': 10.0, 'J': 6.0, 'K': 6.0, 'L': 10.0, 'M': 13.0}
Optimal Duration: 14.0


The minimum total number of weeks requiered to complete all activities is 14. Also, our function yields the optimal starting time for each activity as follows:
<br>
A: 0, B: 0, C: 2, D: 3, E: 3, F: 6, G: 6, H: 6, I: 10, J: 6, K: 6, L: 10, M: 13 